In [6]:
!pip install yfinance beautifulsoup4 plotly pandas



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import yfinance as yf

# Extract Tesla stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

# Display the first few rows
tesla_data.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Use headers to mimic a real browser
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {
    "User-Agent": "Mozilla/5.0"
}
html = requests.get(url, headers=headers).text
soup = BeautifulSoup(html, "html.parser")

# Search for the correct table
tables = soup.find_all("table")
revenue_table = None
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Safety check
if revenue_table is None:
    print("❌ Could not find the Tesla Quarterly Revenue table.")
else:
    tesla_revenue = []
    for row in revenue_table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) == 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            tesla_revenue.append([date, revenue])

    # Create DataFrame
    tesla_revenue_df = pd.DataFrame(tesla_revenue, columns=["Date", "Revenue"])
    tesla_revenue_df["Revenue"] = pd.to_numeric(tesla_revenue_df["Revenue"], errors="coerce")
    tesla_revenue_df = tesla_revenue_df.dropna()
    print(tesla_revenue_df.tail())


          Date  Revenue
58  2010-09-30     31.0
59  2010-06-30     28.0
60  2010-03-31     21.0
62  2009-09-30     46.0
63  2009-06-30     27.0


In [20]:
gamestop = yf.Ticker("GME")
gme_data = gamestop.history(period="max")
gme_data.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607504,7410400,0.0,0.0
2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Use headers to mimic a real browser
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
headers = {
    "User-Agent": "Mozilla/5.0"
}
html = requests.get(url, headers=headers).text
soup = BeautifulSoup(html, "html.parser")

# Search for the correct table
tables = soup.find_all("table")
revenue_table = None
for table in tables:
    if "GameStop Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Safety check
if revenue_table is None:
    print("❌ Could not find the GameStop Quarterly Revenue table.")
else:
    gme_revenue = []
    for row in revenue_table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) == 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            gme_revenue.append([date, revenue])

    # Create DataFrame
    gme_revenue_df = pd.DataFrame(gme_revenue, columns=["Date", "Revenue"])
    gme_revenue_df["Revenue"] = pd.to_numeric(gme_revenue_df["Revenue"], errors="coerce")
    gme_revenue_df = gme_revenue_df.dropna()
    print(gme_revenue_df.tail())


          Date  Revenue
61  2010-01-31     3524
62  2009-10-31     1835
63  2009-07-31     1739
64  2009-04-30     1981
65  2009-01-31     3492


In [36]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

# Set plotly to open in browser
pio.renderers.default = 'browser'

# Convert revenue dates to datetime (naive)
tesla_revenue_df["Date"] = pd.to_datetime(tesla_revenue_df["Date"])

# Make the index of stock data timezone-naive
tesla_data.index = tesla_data.index.tz_localize(None)

# Filter revenue dates to match stock date range
tesla_revenue_df = tesla_revenue_df[
    (tesla_revenue_df["Date"] >= tesla_data.index.min()) &
    (tesla_revenue_df["Date"] <= tesla_data.index.max())
]

# Optional: print to verify
print("Stock range:", tesla_data.index.min(), "-", tesla_data.index.max())
print("Revenue range:", tesla_revenue_df["Date"].min(), "-", tesla_revenue_df["Date"].max())

# Create plot
fig = go.Figure()

# Tesla stock price line
fig.add_trace(go.Scatter(
    x=tesla_data.index,
    y=tesla_data["Close"],
    name="TSLA Stock Price",
    yaxis="y1"
))

# Tesla revenue line
fig.add_trace(go.Scatter(
    x=tesla_revenue_df["Date"],
    y=tesla_revenue_df["Revenue"],
    name="TSLA Revenue",
    yaxis="y2"
))

# Configure layout
fig.update_layout(
    title="Tesla Stock Price vs Quarterly Revenue",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Stock Price (USD)"),
    yaxis2=dict(
        title="Revenue (Million USD)",
        overlaying="y",
        side="right"
    ),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()


Stock range: 2010-06-29 00:00:00 - 2025-07-18 00:00:00
Revenue range: 2010-06-30 00:00:00 - 2025-03-31 00:00:00


In [30]:
import plotly.graph_objects as go
import pandas as pd

gme_data.index = pd.to_datetime(gme_data.index)
gme_revenue_df["Date"] = pd.to_datetime(gme_revenue_df["Date"])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=gme_data.index,
    y=gme_data["Close"],
    name="GME Stock Price",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=gme_revenue_df["Date"],
    y=gme_revenue_df["Revenue"],
    name="GME Revenue",
    yaxis="y2"
))

fig.update_layout(
    title="GameStop Stock Price vs Quarterly Revenue",
    xaxis=dict(title="Date"),
    yaxis=dict(
        title="Stock Price (USD)",
        showgrid=False
    ),
    yaxis2=dict(
        title="Revenue (Million USD)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    legend=dict(x=0.01, y=0.99)
)

fig.show()
